In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
# !ls /home/aistudio/data

!unzip -o data/data183986/valuedata.zip -d data

Archive:  data/data183986/valuedata.zip
  inflating: data/arguments-test.tsv  
  inflating: data/arguments-training.tsv  
  inflating: data/arguments-validation-zhihu.tsv  
  inflating: data/arguments-validation.tsv  
  inflating: data/labels-training.tsv  
  inflating: data/labels-validation-zhihu.tsv  
  inflating: data/labels-validation.tsv  
  inflating: data/level1-labels-training.tsv  
  inflating: data/level1-labels-validation-zhihu.tsv  
  inflating: data/level1-labels-validation.tsv  
  inflating: data/README.md          
  inflating: data/value-categories.json  


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
%cd /home/aistudio/data/

/home/aistudio/data


In [3]:
!ls

arguments-test-nahjalbalagha.tsv  labels-validation.tsv
arguments-test-nyt.tsv		  labels-validation-zhihu.tsv
arguments-test.tsv		  level1-labels-training.tsv
arguments-training.tsv		  level1-labels-validation.tsv
arguments-validation.tsv	  level1-labels-validation-zhihu.tsv
arguments-validation-zhihu.tsv	  README.md
data183986			  value-categories.json
labels-training.tsv


In [4]:
# pandas读取数据集便于分析
import pandas as pd
trainArg = pd.read_csv('arguments-training.tsv', sep='\t', header=0)
validArg = pd.read_csv('arguments-validation.tsv', sep='\t', header=0)
test = pd.read_csv('arguments-test.tsv', sep='\t', header=0)
test2 = pd.read_csv('arguments-test-nahjalbalagha.tsv', sep='\t', header=0)
test3 = pd.read_csv('arguments-test-nyt.tsv', sep='\t', header=0)
zhihuArg = pd.read_csv('arguments-validation-zhihu.tsv', sep='\t', header=0)


trainLal = pd.read_csv('labels-training.tsv', sep='\t', header=0)
validLal = pd.read_csv('labels-validation.tsv', sep='\t', header=0)
zhihuLal = pd.read_csv('labels-validation-zhihu.tsv', sep='\t', header=0)

In [5]:
# dataframe转列表list
def dataframetoList(X):
    train_data = np.array(X)#np.ndarray()
    train_x_list=train_data.tolist()#list
    return train_x_list

In [6]:
import numpy as np
trainArgList = dataframetoList(trainArg)
validArgList = dataframetoList(validArg)
zhihuArgList = dataframetoList(zhihuArg)
testList = dataframetoList(test)
test2List = dataframetoList(test2)
test3List = dataframetoList(test3)

In [7]:
def insertSentence(dataList):
    for x in range(0, len(dataList)):
        sentence = "Imagine someone is arguing " + str(dataList[x][2]) + " \"" + str(dataList[x][1]) + "\" " + "by saying: \"" + str(dataList[x][3]) + "\"."
        dataList[x].append(sentence)
    return dataList

In [8]:
trainArgs = insertSentence(trainArgList)
validArgs = insertSentence(validArgList)
zhihuArgs = insertSentence(zhihuArgList)
testLists = insertSentence(testList)
test2Lists = insertSentence(test2List)
test3Lists = insertSentence(test3List)

In [9]:
# list 转dataframe
trainArgD = pd.DataFrame(trainArgs)
trainArgD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]

validArgD = pd.DataFrame(validArgs)
validArgD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]

zhihuArgD = pd.DataFrame(zhihuArgs)
zhihuArgD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]

testD = pd.DataFrame(testLists)
testD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]

test2D = pd.DataFrame(test2Lists)
test2D.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]

test3D = pd.DataFrame(test3Lists)
test3D.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]



In [10]:
trainArgD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
validArgD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
zhihuArgD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
testD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
test2D.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
test3D.drop(columns=["Conclusion","Stance","Premise"],inplace=True)

In [11]:
def labelProcess(datalabel):   
    labels = []
    for i in range(len(datalabel)):
        label = list(datalabel.iloc[i])[1:]
        labels.append(label)
    #labels[0]
    datalabel["labels"] = labels
    deletelist = list(datalabel.columns)[1:-1]
    datalabel.drop(columns=deletelist,inplace=True)
    return datalabel

In [12]:
trainLal = labelProcess(trainLal)    
validLal = labelProcess(validLal)    
zhihuLal = labelProcess(zhihuLal)

In [13]:
train_merge = trainArgD.merge(trainLal,on='Argument ID')
valid_merge = validArgD.merge(validLal,on='Argument ID')
zhihu_merge = zhihuArgD.merge(zhihuLal,on='Argument ID')

In [14]:
train_merge.to_csv('train.tsv',columns=train_merge.columns.tolist(),
            sep='\t',
            index=False)

valid_merge.to_csv('dev.tsv',columns=train_merge.columns.tolist(),
            sep='\t',
            index=False)
zhihu_merge.to_csv('zhihu_validation.tsv',columns=zhihu_merge.columns.tolist(),
            sep='\t',
            index=False)
testD.to_csv('test.tsv',columns=testD.columns.tolist(),
            sep='\t',
            index=False)
test2D.to_csv('test2.tsv',columns=test2D.columns.tolist(),
            sep='\t',
            index=False)
test3D.to_csv('test3.tsv',columns=test3D.columns.tolist(),
            sep='\t',
            index=False)